# Overview

This Notebook demonstrates how to interact with the test app API.  Remember that you can see the documented endpoints at http://localhost:8000/docs.  Note that the `base_url` in our client says `backend:8000`, which is the right domain for code in the Jupyter container to communicate to the test app in the same docker network.  Your browser is not in that docker network, which is why it's navigating to `localhost:8000`.

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import httpx

client = httpx.Client(base_url="http://backend:8000", follow_redirects=True)
client

<IPython.core.display.Javascript object>

# Login

Before we can do anything with the Todo endpoints we need to log in

In [3]:
resp = client.get("/me")
resp

<Response [401 Unauthorized]>

<IPython.core.display.Javascript object>

In [4]:
resp.json()

{'detail': 'Not authenticated'}

<IPython.core.display.Javascript object>

In [5]:
endpoint = "/auth/login"
data = {"username": "user1", "password": "pass"}

resp = client.post(endpoint, data=data)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [6]:
resp.json()

{'access_token': 'user1', 'token_type': 'bearer'}

<IPython.core.display.Javascript object>

In [7]:
client.headers["Authorization"] = "bearer user1"
resp = client.get("/me")
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [8]:
resp.json()

{'id': '9feeed1d-833e-4fd1-a3c0-0e0980688561',
 'created_at': '2021-10-29T13:38:04.073930+00:00',
 'name': 'user1'}

<IPython.core.display.Javascript object>

In [9]:
resp.request.headers

Headers({'host': 'backend:8000', 'accept': '*/*', 'accept-encoding': 'gzip, deflate, br', 'connection': 'keep-alive', 'user-agent': 'python-httpx/0.20.0', 'authorization': '[secure]'})

<IPython.core.display.Javascript object>

# Get todos

In [10]:
resp = client.get("/todo")
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [11]:
resp.json()

[{'id': '2f262a8f-58e5-4f89-aff1-f9ac19d9efe8',
  'created_at': '2021-10-29T13:38:04.073930+00:00',
  'title': 'Note 4',
  'content': 'Reminder: make more Notes'},
 {'id': 'c8721eab-8e77-42fd-ae16-f2047d4e3149',
  'created_at': '2021-10-29T13:38:11.955925+00:00',
  'title': 'string',
  'content': 'string'},
 {'id': 'a337136c-3eea-4f6d-bb84-c16c7ea2bea5',
  'created_at': '2021-10-29T13:38:04.073930+00:00',
  'title': 'Note 1',
  'content': 'My first Note'},
 {'id': '9f3abbc9-cf9d-4576-9c8a-d6c5747af157',
  'created_at': '2021-10-29T13:38:04.073930+00:00',
  'title': 'Note 3',
  'content': 'So many Notes'},
 {'id': '9d667b2c-0547-4bac-9b86-714fced1023b',
  'created_at': '2021-10-29T13:38:04.073930+00:00',
  'title': 'Note 2',
  'content': 'Edit this Note'}]

<IPython.core.display.Javascript object>

In [12]:
# note it followed a redirect!
resp.history

[<Response [307 Temporary Redirect]>]

<IPython.core.display.Javascript object>

In [13]:
resp.history[0].headers

Headers({'date': 'Fri, 29 Oct 2021 13:38:22 GMT', 'server': 'uvicorn', 'location': 'http://backend:8000/todo/', 'transfer-encoding': 'chunked'})

<IPython.core.display.Javascript object>

# Make a todo

In [14]:
data = {"title": "programmatic Note", "content": "made from python"}
resp = client.post("/todo", json=data)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [15]:
resp.request.read()

b'{"title": "programmatic Note", "content": "made from python"}'

<IPython.core.display.Javascript object>

In [16]:
resp.json()

{'id': '79ac12f9-d903-44c2-9ff0-11c3ea6f9f3d',
 'created_at': '2021-10-29T13:38:30.459175+00:00',
 'title': 'programmatic Note',
 'content': 'made from python'}

<IPython.core.display.Javascript object>

In [17]:
# check that it exists in the "get all" now
resp = client.get("/todo")
resp.json()

[{'id': '2f262a8f-58e5-4f89-aff1-f9ac19d9efe8',
  'created_at': '2021-10-29T13:38:04.073930+00:00',
  'title': 'Note 4',
  'content': 'Reminder: make more Notes'},
 {'id': 'c8721eab-8e77-42fd-ae16-f2047d4e3149',
  'created_at': '2021-10-29T13:38:11.955925+00:00',
  'title': 'string',
  'content': 'string'},
 {'id': 'a337136c-3eea-4f6d-bb84-c16c7ea2bea5',
  'created_at': '2021-10-29T13:38:04.073930+00:00',
  'title': 'Note 1',
  'content': 'My first Note'},
 {'id': '9f3abbc9-cf9d-4576-9c8a-d6c5747af157',
  'created_at': '2021-10-29T13:38:04.073930+00:00',
  'title': 'Note 3',
  'content': 'So many Notes'},
 {'id': '9d667b2c-0547-4bac-9b86-714fced1023b',
  'created_at': '2021-10-29T13:38:04.073930+00:00',
  'title': 'Note 2',
  'content': 'Edit this Note'},
 {'id': '79ac12f9-d903-44c2-9ff0-11c3ea6f9f3d',
  'created_at': '2021-10-29T13:38:30.459175+00:00',
  'title': 'programmatic Note',
  'content': 'made from python'}]

<IPython.core.display.Javascript object>

# Update a todo

In [18]:
todo = resp.json()[0]
todo

{'id': '2f262a8f-58e5-4f89-aff1-f9ac19d9efe8',
 'created_at': '2021-10-29T13:38:04.073930+00:00',
 'title': 'Note 4',
 'content': 'Reminder: make more Notes'}

<IPython.core.display.Javascript object>

In [19]:
_id = todo["id"]
endpoint = f"/todo/{_id}"

data = {"content": "updated by python"}
resp = client.put(endpoint, data=data)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [20]:
resp.json()

{'id': '2f262a8f-58e5-4f89-aff1-f9ac19d9efe8',
 'created_at': '2021-10-29T13:38:04.073930+00:00',
 'title': 'Note 4',
 'content': 'updated by python'}

<IPython.core.display.Javascript object>

# Delete a todo

In [21]:
resp = client.delete(endpoint)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [22]:
# calling it again shows 404, already deleted
resp = client.delete(endpoint)
resp

<Response [404 Not Found]>

<IPython.core.display.Javascript object>